# 任务六

In [ ]:
# 导入数据
import pandas as pd
df1 = pd.read_excel(r'D:\研究生资料\玛氏数据\5-22\chengdu_521.xlsx')
df_POI = pd.read_excel(r'D:\研究生资料\玛氏数据\chengdu_poi_category.xlsx')

## 一、有卖巧克力（平均订单时间<3个月）并且卖口香糖（平均下单时间>6个月）的门店对应前5个POI 出现次数

1)	求出卖巧克力的门店

In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

2) 求出巧克力平均下单时间小于3个月的门店

In [ ]:
# 把Chocolate的那部分分成下单周期小于3个月和下达周期大于3个月两类
Cstore_id_da3 = [] # 用来储存买Chocolate的那部分下单周期大于3个月的'store_code'
Cstore_id_xiao3 = [] # 用来储存买Chocolate的那部分下单周期小于等于3个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da3.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            Cstore_id_da3.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao3.append(Chocolate_storeid[i]) 

3) 求出（2）中门店中卖口香糖的门店

In [ ]:
# 在Cstore_id_xiao3里面筛选出卖口香糖的门店；
demo1=df1.loc[df1['store_code'].isin(Cstore_id_xiao3)]
demo2=demo1.loc[demo1['ProductCategory']=='Chewing/Bubble Gum']
Chox3_Che=list(set(list(demo2['store_code'])))

4) 求出（3）中口香糖平均下单时间大于6个月的门店；

In [ ]:
# 将Chox3_Che划分为口香糖平均下单时间>6个月
CC_da6=[]
CC_xiao6=[]
for i in range(len(Chox3_Che)):
    
    demo0 = df1.loc[df1['store_code']==Chox3_Che[i]]
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chewing/Bubble Gum']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    print('day',day)
    #计算订单个数
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    print('order_sum',order_sum)
    
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        CC_da6.append(Chox3_Che[i])
    else:
        interval = day/(order_sum-1)
        if interval>180:
            CC_da6.append(Chox3_Che[i])
        else:
            CC_xiao6.append(Chox3_Che[i]) 

5) 求出（4）中门店对应的POI，去重、去充电宝，按照距离有小到大保留前五个POI；

In [ ]:
# CC_da6对应的poi
CC_da6_POI=df_POI[df_POI['storecode'].isin(CC_da6)]
# 去充电宝
CC_da6_POI=CC_da6_POI.drop(CC_da6_POI[CC_da6_POI['s_cat_value']=='充电宝'].index)
# 对每个门店的poi去重（相同POI留下距离近的那个POI）
CC_da6_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k1_1 = CC_da6_POI.groupby(['storecode','s_cat_id']).head(5)

6) 求出（5）中每个POI的出现次数

In [ ]:
# 在k1_1的基础上，求POI出现次数
k1_1['nums']=0
uu_1=list(k1_1.groupby(['s_cat_id']).count()['nums'].index) # 表示s_cat_id
uu_2=list(k1_1.groupby(['s_cat_id']).count()['nums'].values) # 表示出现次数
uu_3=[]
uu_4=[]
uu_5=[]
uu_6=[]
uu_7=[]
# 'b_cat_id','b_cat_value','m_cat_id','m_cat_value'
for i in range(len(uu_1)):
    f1=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'b_cat_id'])[0]
    f2=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'b_cat_value'])[0]
    f3=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'m_cat_id'])[0]
    f4=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'m_cat_value'])[0]
    f5=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'s_cat_value'])[0]
    uu_3.append(f1)
    uu_4.append(f2)
    uu_5.append(f3)
    uu_6.append(f4)
    uu_7.append(f5)

data3={
    'b_cat_id':uu_3,
    'b_cat_value':uu_4,
    'm_cat_id':uu_5,
    'm_cat_value':uu_6,
    's_cat_id':uu_1,
    's_cat_value':uu_7,
    '出现次数':uu_2
    
}
k1_2=pd.DataFrame(data3)
k1_2.to_excel(r'C:\Users\Administrator\Desktop\任务六（一）.xlsx')

## 二、有卖巧克力（平均订单时间<3个月）并且卖口香糖（平均下单时间>6个月）的门店对应前10个POI 出现次数

1)	求出卖巧克力的门店

In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

2)	求出巧克力平均下单时间小于3个月的门店

In [ ]:
# 把Chocolate的那部分分成下单周期小于3个月和下达周期大于3个月两类
Cstore_id_da3 = [] # 用来储存买Chocolate的那部分下单周期大于3个月的'store_code'
Cstore_id_xiao3 = [] # 用来储存买Chocolate的那部分下单周期小于等于3个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da3.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            Cstore_id_da3.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao3.append(Chocolate_storeid[i]) 

3)	求出（2）中门店中卖口香糖的门店

In [ ]:
# 在Cstore_id_xiao3里面筛选出卖口香糖的门店；
demo1=df1.loc[df1['store_code'].isin(Cstore_id_xiao3)]
demo2=demo1.loc[demo1['ProductCategory']=='Chewing/Bubble Gum']
Chox3_Che=list(set(list(demo2['store_code'])))

4)	求出（3）中口香糖平均下单时间大于6个月的门店；

In [ ]:
# 将Chox3_Che划分为口香糖平均下单时间>6个月
CC_da6=[]
CC_xiao6=[]
for i in range(len(Chox3_Che)):
    
    demo0 = df1.loc[df1['store_code']==Chox3_Che[i]]
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chewing/Bubble Gum']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    print('day',day)
    #计算订单个数
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    print('order_sum',order_sum)
    
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        CC_da6.append(Chox3_Che[i])
    else:
        interval = day/(order_sum-1)
        if interval>180:
            CC_da6.append(Chox3_Che[i])
        else:
            CC_xiao6.append(Chox3_Che[i]) 

5)	求出（4）中门店对应的POI，去重、去充电宝，按照距离有小到大保留前十个POI；

In [ ]:
# CC_da6对应的poi
CC_da6_POI=df_POI[df_POI['storecode'].isin(CC_da6)]
# 去充电宝
CC_da6_POI=CC_da6_POI.drop(CC_da6_POI[CC_da6_POI['s_cat_value']=='充电宝'].index)
# 对每个门店的poi去重（相同POI留下距离近的那个POI）
CC_da6_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k2_1 = CC_da6_POI.groupby(['storecode','s_cat_id']).head(10)

6)	求出（5）中每个POI的出现次数

In [ ]:
# 在k2_1的基础上，求POI出现次数
k2_1['nums']=0
uu_1=list(k2_1.groupby(['s_cat_id']).count()['nums'].index) # 表示s_cat_id
uu_2=list(k2_1.groupby(['s_cat_id']).count()['nums'].values) # 表示出现次数
uu_3=[]
uu_4=[]
uu_5=[]
uu_6=[]
uu_7=[]
# 'b_cat_id','b_cat_value','m_cat_id','m_cat_value'
for i in range(len(uu_1)):
    f1=list(k2_1.loc[k2_1['s_cat_id']==uu_1[i],'b_cat_id'])[0]
    f2=list(k2_1.loc[k2_1['s_cat_id']==uu_1[i],'b_cat_value'])[0]
    f3=list(k2_1.loc[k2_1['s_cat_id']==uu_1[i],'m_cat_id'])[0]
    f4=list(k2_1.loc[k2_1['s_cat_id']==uu_1[i],'m_cat_value'])[0]
    f5=list(k2_1.loc[k2_1['s_cat_id']==uu_1[i],'s_cat_value'])[0]
    uu_3.append(f1)
    uu_4.append(f2)
    uu_5.append(f3)
    uu_6.append(f4)
    uu_7.append(f5)

data3={
    'b_cat_id':uu_3,
    'b_cat_value':uu_4,
    'm_cat_id':uu_5,
    'm_cat_value':uu_6,
    's_cat_id':uu_1,
    's_cat_value':uu_7,
    '出现次数':uu_2
    
}
k2_2=pd.DataFrame(data3)
k2_2.to_excel(r'C:\Users\Administrator\Desktop\任务六（二）.xlsx')

## 三、有卖巧克力（平均订单时间<3个月）并且卖口香糖（平均下单时间>3个月）的门店对应前5个POI 出现次数

1)	求出卖巧克力的门店

In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

2)	求出巧克力平均下单时间小于3个月的门店

In [ ]:
# 把Chocolate的那部分分成下单周期小于3个月和下达周期大于3个月两类
Cstore_id_da3 = [] # 用来储存买Chocolate的那部分下单周期大于3个月的'store_code'
Cstore_id_xiao3 = [] # 用来储存买Chocolate的那部分下单周期小于等于3个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da3.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            Cstore_id_da3.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao3.append(Chocolate_storeid[i]) 

3)	求出（2）中门店中卖口香糖的门店

In [ ]:
# 在Cstore_id_xiao3里面筛选出卖口香糖的门店；
demo1=df1.loc[df1['store_code'].isin(Cstore_id_xiao3)]
demo2=demo1.loc[demo1['ProductCategory']=='Chewing/Bubble Gum']
Chox3_Che=list(set(list(demo2['store_code'])))

4)	求出（3）中口香糖平均下单时间大于3个月的门店；

In [ ]:
# 将Chox3_Che划分为口香糖平均下单时间>3个月
CC_da3=[]
CC_xiao3=[]
for i in range(len(Chox3_Che)):
    
    demo0 = df1.loc[df1['store_code']==Chox3_Che[i]]
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chewing/Bubble Gum']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    print('day',day)
    #计算订单个数
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    print('order_sum',order_sum)
    
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        CC_da3.append(Chox3_Che[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            CC_da3.append(Chox3_Che[i])
        else:
            CC_xiao3.append(Chox3_Che[i]) 

5)	求出（4）中门店对应的POI，去重、去充电宝，按照距离有小到大保留前五个POI；

In [ ]:
# CC_da3对应的poi
CC_da3_POI=df_POI[df_POI['storecode'].isin(CC_da3)]
# 去充电宝
CC_da3_POI=CC_da3_POI.drop(CC_da3_POI[CC_da3_POI['s_cat_value']=='充电宝'].index)
# 对每个门店的poi去重（相同POI留下距离近的那个POI）
CC_da3_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k3_1 = CC_da3_POI.groupby(['storecode','s_cat_id']).head(5)

6)	求出（5）中每个POI的出现次数

In [ ]:
# 在k3_1的基础上，求POI出现次数
k3_1['nums']=0
uu_1=list(k3_1.groupby(['s_cat_id']).count()['nums'].index) # 表示s_cat_id
uu_2=list(k3_1.groupby(['s_cat_id']).count()['nums'].values) # 表示出现次数
uu_3=[]
uu_4=[]
uu_5=[]
uu_6=[]
uu_7=[]
# 'b_cat_id','b_cat_value','m_cat_id','m_cat_value'
for i in range(len(uu_1)):
    f1=list(k3_1.loc[k3_1['s_cat_id']==uu_1[i],'b_cat_id'])[0]
    f2=list(k3_1.loc[k3_1['s_cat_id']==uu_1[i],'b_cat_value'])[0]
    f3=list(k3_1.loc[k3_1['s_cat_id']==uu_1[i],'m_cat_id'])[0]
    f4=list(k3_1.loc[k3_1['s_cat_id']==uu_1[i],'m_cat_value'])[0]
    f5=list(k3_1.loc[k3_1['s_cat_id']==uu_1[i],'s_cat_value'])[0]
    uu_3.append(f1)
    uu_4.append(f2)
    uu_5.append(f3)
    uu_6.append(f4)
    uu_7.append(f5)

data3={
    'b_cat_id':uu_3,
    'b_cat_value':uu_4,
    'm_cat_id':uu_5,
    'm_cat_value':uu_6,
    's_cat_id':uu_1,
    's_cat_value':uu_7,
    '出现次数':uu_2
    
}
k3_2=pd.DataFrame(data3)
k3_2.to_excel(r'C:\Users\Administrator\Desktop\任务六（三）.xlsx')

## 四、有卖巧克力（平均订单时间<3个月）并且卖口香糖（平均下单时间>3个月）的门店对应前10个POI出现次数

1)	求出卖巧克力的门店

In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

2)	求出巧克力平均下单时间小于3个月的门店

In [ ]:
# 把Chocolate的那部分分成下单周期小于3个月和下达周期大于3个月两类
Cstore_id_da3 = [] # 用来储存买Chocolate的那部分下单周期大于3个月的'store_code'
Cstore_id_xiao3 = [] # 用来储存买Chocolate的那部分下单周期小于等于3个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da3.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            Cstore_id_da3.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao3.append(Chocolate_storeid[i]) 

3)	求出（2）中门店中卖口香糖的门店

In [ ]:
# 在Cstore_id_xiao3里面筛选出卖口香糖的门店；
demo1=df1.loc[df1['store_code'].isin(Cstore_id_xiao3)]
demo2=demo1.loc[demo1['ProductCategory']=='Chewing/Bubble Gum']
Chox3_Che=list(set(list(demo2['store_code'])))

4)	求出（3）中口香糖平均下单时间大于3个月的门店；

In [ ]:
# 将Chox3_Che划分为口香糖平均下单时间>3个月
CC_da3=[]
CC_xiao3=[]
for i in range(len(Chox3_Che)):
    
    demo0 = df1.loc[df1['store_code']==Chox3_Che[i]]
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chewing/Bubble Gum']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    print('day',day)
    #计算订单个数
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    print('order_sum',order_sum)
    
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        CC_da3.append(Chox3_Che[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            CC_da3.append(Chox3_Che[i])
        else:
            CC_xiao3.append(Chox3_Che[i]) 

5)	求出（4）中门店对应的POI，去重、去充电宝，按照距离有小到大保留前十个POI；

In [ ]:
# CC_da3对应的poi
CC_da3_POI=df_POI[df_POI['storecode'].isin(CC_da3)]
# 去充电宝
CC_da3_POI=CC_da3_POI.drop(CC_da3_POI[CC_da3_POI['s_cat_value']=='充电宝'].index)
# 对每个门店的poi去重（相同POI留下距离近的那个POI）
CC_da3_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k4_1 = CC_da3_POI.groupby(['storecode','s_cat_id']).head(10)

6)	求出（5）中每个POI的出现次数

In [ ]:
# 在k3_1的基础上，求POI出现次数
k4_1['nums']=0
uu_1=list(k4_1.groupby(['s_cat_id']).count()['nums'].index) # 表示s_cat_id
uu_2=list(k4_1.groupby(['s_cat_id']).count()['nums'].values) # 表示出现次数
uu_3=[]
uu_4=[]
uu_5=[]
uu_6=[]
uu_7=[]
# 'b_cat_id','b_cat_value','m_cat_id','m_cat_value'
for i in range(len(uu_1)):
    f1=list(k4_1.loc[k4_1['s_cat_id']==uu_1[i],'b_cat_id'])[0]
    f2=list(k4_1.loc[k4_1['s_cat_id']==uu_1[i],'b_cat_value'])[0]
    f3=list(k4_1.loc[k4_1['s_cat_id']==uu_1[i],'m_cat_id'])[0]
    f4=list(k4_1.loc[k4_1['s_cat_id']==uu_1[i],'m_cat_value'])[0]
    f5=list(k4_1.loc[k4_1['s_cat_id']==uu_1[i],'s_cat_value'])[0]
    uu_3.append(f1)
    uu_4.append(f2)
    uu_5.append(f3)
    uu_6.append(f4)
    uu_7.append(f5)

data3={
    'b_cat_id':uu_3,
    'b_cat_value':uu_4,
    'm_cat_id':uu_5,
    'm_cat_value':uu_6,
    's_cat_id':uu_1,
    's_cat_value':uu_7,
    '出现次数':uu_2
    
}
k4_2=pd.DataFrame(data3)
k4_2.to_excel(r'C:\Users\Administrator\Desktop\任务六（四）.xlsx')

## 五、有卖巧克力（平均订单时间<3个月）并且卖口香糖（平均下单时间>6个月）的门店对应前5个POI的巧克力累计销量

1)	求出卖巧克力的门店

In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

2)	求出巧克力平均下单时间小于3个月的门店

In [ ]:
# 把Chocolate的那部分分成下单周期小于3个月和下达周期大于3个月两类
Cstore_id_da3 = [] # 用来储存买Chocolate的那部分下单周期大于3个月的'store_code'
Cstore_id_xiao3 = [] # 用来储存买Chocolate的那部分下单周期小于等于3个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da3.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            Cstore_id_da3.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao3.append(Chocolate_storeid[i]) 

3)	求出（2）中门店中卖口香糖的门店

In [ ]:
# 在Cstore_id_xiao3里面筛选出卖口香糖的门店；
demo1=df1.loc[df1['store_code'].isin(Cstore_id_xiao3)]
demo2=demo1.loc[demo1['ProductCategory']=='Chewing/Bubble Gum']
Chox3_Che=list(set(list(demo2['store_code'])))

4)	求出（3）中口香糖平均下单时间大于6个月的门店；

In [ ]:
# 将Chox3_Che划分为口香糖平均下单时间>6个月
CC_da6=[]
CC_xiao6=[]
for i in range(len(Chox3_Che)):
    
    demo0 = df1.loc[df1['store_code']==Chox3_Che[i]]
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chewing/Bubble Gum']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    print('day',day)
    #计算订单个数
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    print('order_sum',order_sum)
    
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        CC_da6.append(Chox3_Che[i])
    else:
        interval = day/(order_sum-1)
        if interval>180:
            CC_da6.append(Chox3_Che[i])
        else:
            CC_xiao6.append(Chox3_Che[i]) 

5)	求出（4）中门店对应的POI，去重、去充电宝，按照距离有小到大保留前5个POI；

In [ ]:
# CC_da6对应的poi
CC_da6_POI=df_POI[df_POI['storecode'].isin(CC_da6)]
# 去充电宝
CC_da6_POI=CC_da6_POI.drop(CC_da6_POI[CC_da6_POI['s_cat_value']=='充电宝'].index)
# 对每个门店的poi去重（相同POI留下距离近的那个POI）
CC_da6_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k5_1 = CC_da6_POI.groupby(['storecode','s_cat_id']).head(5)

6)	求出（3）中每个门店的巧克力销量（case-ordered）

In [ ]:
# 在Chox3_Che里面筛选出卖巧克力的门店；
demo1=df1.loc[df1['store_code'].isin(Chox3_Che)]
demo2=demo1.loc[demo1['ProductCategory']=='Chocolate']
Chox3_Che_Cho=list(set(list(demo2['store_code'])))
# 门店巧克力销售加总
cho_sum=[] 
for i in range(len(Chox3_Che_Cho)):
    demo3 = df1.loc[df1['store_code']==Chox3_Che_Cho[i]]
    sum_1 = demo3.loc[demo3['ProductCategory']=='Chocolate','case_ordered'].sum()
    cho_sum.append(sum_1)

7)	将（6）中的每个门店销量赋值到（5）中每个门店对应的POI上；

In [ ]:
# k5_1赋上销量信息
# 构建门店编号及其销量信息的表
data3={
    'Chox3_Che_Cho':Chox3_Che_Cho,
    'cho_sum':cho_sum
}
data3=pd.DataFrame(data3)
data3.sort_values(by='Chox3_Che_Cho',inplace=True)

k5_1.sort_values(by='storecode',inplace=True)

sum_2=list(data3['cho_sum'])
s_id =list(data3['Chox3_Che_Cho'])

sum_3 =[]
for i in range(len(s_id)):
    cc=len(k5_1.loc[k5_1['storecode']==s_id[i]])
    for j in range(cc):
        sum_3.append(sum_2[i])

# print(len(sum_3))
# print(len(k3_1))

k5_1['总销量']=sum_3

fff=k5_1[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

8)	求每个POI的销量累计值，按照销量由大到小排序；

In [ ]:
k5_4 = fff.groupby(['s_cat_id','s_cat_value'])['总销量'].sum()
k5_4 = k5_4.reset_index()
k5_4.sort_values(['总销量'],ascending=[0],inplace=True)
tt= list(k5_4['s_cat_id'])
r1=[]
r2=[]
r3=[]
r4=[]
for i in range(len(tt)):
    r1.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_id'])))[0])
    r2.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_value'])))[0])
    r3.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_id'])))[0])
    r4.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_value'])))[0])
k5_4['b_cat_id']=r1
k5_4['b_cat_value']=r2
k5_4['m_cat_id']=r3
k5_4['m_cat_value']=r4

k5_4=k5_4[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

k5_4.to_excel(r'C:\Users\Administrator\Desktop\任务六（五）.xlsx')

## 六、有卖巧克力（平均订单时间<3个月）并且卖口香糖（平均下单时间>6个月）的门店对应前10个POI的巧克力累计销量

1)	求出卖巧克力的门店

In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

2)	求出巧克力平均下单时间小于3个月的门店

In [ ]:
# 把Chocolate的那部分分成下单周期小于3个月和下达周期大于3个月两类
Cstore_id_da3 = [] # 用来储存买Chocolate的那部分下单周期大于3个月的'store_code'
Cstore_id_xiao3 = [] # 用来储存买Chocolate的那部分下单周期小于等于3个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da3.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            Cstore_id_da3.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao3.append(Chocolate_storeid[i]) 

3)	求出（2）中门店中卖口香糖的门店

In [ ]:
# 在Cstore_id_xiao3里面筛选出卖口香糖的门店；
demo1=df1.loc[df1['store_code'].isin(Cstore_id_xiao3)]
demo2=demo1.loc[demo1['ProductCategory']=='Chewing/Bubble Gum']
Chox3_Che=list(set(list(demo2['store_code'])))

4)	求出（3）中口香糖平均下单时间大于6个月的门店；

In [ ]:
# 将Chox3_Che划分为口香糖平均下单时间>6个月
CC_da6=[]
CC_xiao6=[]
for i in range(len(Chox3_Che)):
    
    demo0 = df1.loc[df1['store_code']==Chox3_Che[i]]
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chewing/Bubble Gum']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    print('day',day)
    #计算订单个数
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    print('order_sum',order_sum)
    
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        CC_da6.append(Chox3_Che[i])
    else:
        interval = day/(order_sum-1)
        if interval>180:
            CC_da6.append(Chox3_Che[i])
        else:
            CC_xiao6.append(Chox3_Che[i]) 

5)	求出（4）中门店对应的POI，去重、去充电宝，按照距离有小到大保留前十个POI；

In [ ]:
# CC_da6对应的poi
CC_da6_POI=df_POI[df_POI['storecode'].isin(CC_da6)]
# 去充电宝
CC_da6_POI=CC_da6_POI.drop(CC_da6_POI[CC_da6_POI['s_cat_value']=='充电宝'].index)
# 对每个门店的poi去重（相同POI留下距离近的那个POI）
CC_da6_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k6_1 = CC_da6_POI.groupby(['storecode','s_cat_id']).head(10)

6) 求出（3）中每个门店的巧克力销量（case-ordered）

In [ ]:
# 在Chox3_Che里面筛选出卖巧克力的门店；
demo1=df1.loc[df1['store_code'].isin(Chox3_Che)]
demo2=demo1.loc[demo1['ProductCategory']=='Chocolate']
Chox3_Che_Cho=list(set(list(demo2['store_code'])))
# 门店巧克力销售加总
cho_sum=[] 
for i in range(len(Chox3_Che_Cho)):
    demo3 = df1.loc[df1['store_code']==Chox3_Che_Cho[i]]
    sum_1 = demo3.loc[demo3['ProductCategory']=='Chocolate','case_ordered'].sum()
    cho_sum.append(sum_1)

7)	将（6）中的每个门店销量赋值到（5）中每个门店对应的POI上；

In [ ]:
# k6_1赋上销量信息
# 构建门店编号及其销量信息的表
data3={
    'Chox3_Che_Cho':Chox3_Che_Cho,
    'cho_sum':cho_sum
}
data3=pd.DataFrame(data3)
data3.sort_values(by='Chox3_Che_Cho',inplace=True)

k6_1.sort_values(by='storecode',inplace=True)

sum_2=list(data3['cho_sum'])
s_id =list(data3['Chox3_Che_Cho'])

sum_3 =[]
for i in range(len(s_id)):
    cc=len(k6_1.loc[k6_1['storecode']==s_id[i]])
    for j in range(cc):
        sum_3.append(sum_2[i])

# print(len(sum_3))
# print(len(k3_1))

k6_1['总销量']=sum_3

fff=k6_1[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

8)	求每个POI的销量累计值，按照销量由大到小排序；

In [ ]:
k6_4 = fff.groupby(['s_cat_id','s_cat_value'])['总销量'].sum()
k6_4 = k6_4.reset_index()
k6_4.sort_values(['总销量'],ascending=[0],inplace=True)
tt= list(k6_4['s_cat_id'])
r1=[]
r2=[]
r3=[]
r4=[]
for i in range(len(tt)):
    r1.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_id'])))[0])
    r2.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_value'])))[0])
    r3.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_id'])))[0])
    r4.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_value'])))[0])
k6_4['b_cat_id']=r1
k6_4['b_cat_value']=r2
k6_4['m_cat_id']=r3
k6_4['m_cat_value']=r4

k6_4=k6_4[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

k6_4.to_excel(r'C:\Users\Administrator\Desktop\任务六（六）.xlsx')

## 七、有卖巧克力（平均订单时间<3个月）并且卖口香糖（平均下单时间>3个月）的门店对应前5个POI的巧克力累计销量

1)	求出卖巧克力的门店

In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

2)	求出巧克力平均下单时间小于3个月的门店

In [ ]:
# 把Chocolate的那部分分成下单周期小于3个月和下达周期大于3个月两类
Cstore_id_da3 = [] # 用来储存买Chocolate的那部分下单周期大于3个月的'store_code'
Cstore_id_xiao3 = [] # 用来储存买Chocolate的那部分下单周期小于等于3个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da3.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            Cstore_id_da3.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao3.append(Chocolate_storeid[i]) 

3)	求出（2）中门店中卖口香糖的门店

In [ ]:
# 在Cstore_id_xiao3里面筛选出卖口香糖的门店；
demo1=df1.loc[df1['store_code'].isin(Cstore_id_xiao3)]
demo2=demo1.loc[demo1['ProductCategory']=='Chewing/Bubble Gum']
Chox3_Che=list(set(list(demo2['store_code'])))

4)	求出（3）中口香糖平均下单时间大于3个月的门店；

In [ ]:
# 将Chox3_Che划分为口香糖平均下单时间>3个月
CC_da3=[]
CC_xiao3=[]
for i in range(len(Chox3_Che)):
    
    demo0 = df1.loc[df1['store_code']==Chox3_Che[i]]
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chewing/Bubble Gum']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    print('day',day)
    #计算订单个数
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    print('order_sum',order_sum)
    
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        CC_da3.append(Chox3_Che[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            CC_da3.append(Chox3_Che[i])
        else:
            CC_xiao3.append(Chox3_Che[i]) 

5)	求出（4）中门店对应的POI，去重、去充电宝，按照距离有小到大保留前5个POI；

In [ ]:
# CC_da3对应的poi
CC_da3_POI=df_POI[df_POI['storecode'].isin(CC_da3)]
# 去充电宝
CC_da3_POI=CC_da3_POI.drop(CC_da3_POI[CC_da3_POI['s_cat_value']=='充电宝'].index)
# 对每个门店的poi去重（相同POI留下距离近的那个POI）
CC_da3_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k7_1 = CC_da3_POI.groupby(['storecode','s_cat_id']).head(5)

6)	求出（3）中每个门店的巧克力销量（case-ordered）

In [ ]:
# 在Chox3_Che里面筛选出卖巧克力的门店；
demo1=df1.loc[df1['store_code'].isin(Chox3_Che)]
demo2=demo1.loc[demo1['ProductCategory']=='Chocolate']
Chox3_Che_Cho=list(set(list(demo2['store_code'])))
# 门店巧克力销售加总
cho_sum=[] 
for i in range(len(Chox3_Che_Cho)):
    demo3 = df1.loc[df1['store_code']==Chox3_Che_Cho[i]]
    sum_1 = demo3.loc[demo3['ProductCategory']=='Chocolate','case_ordered'].sum()
    cho_sum.append(sum_1)

7)	将（6）中的每个门店销量赋值到（5）中每个门店对应的POI上；

In [ ]:
# k7_1赋上销量信息
# 构建门店编号及其销量信息的表
data3={
    'Chox3_Che_Cho':Chox3_Che_Cho,
    'cho_sum':cho_sum
}
data3=pd.DataFrame(data3)
data3.sort_values(by='Chox3_Che_Cho',inplace=True)

k7_1.sort_values(by='storecode',inplace=True)

sum_2=list(data3['cho_sum'])
s_id =list(data3['Chox3_Che_Cho'])

sum_3 =[]
for i in range(len(s_id)):
    cc=len(k7_1.loc[k7_1['storecode']==s_id[i]])
    for j in range(cc):
        sum_3.append(sum_2[i])

# print(len(sum_3))
# print(len(k3_1))

k7_1['总销量']=sum_3

fff=k7_1[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

8)	求每个POI的销量累计值，按照销量由大到小排序；

In [ ]:
k7_4 = fff.groupby(['s_cat_id','s_cat_value'])['总销量'].sum()
k7_4 = k7_4.reset_index()
k7_4.sort_values(['总销量'],ascending=[0],inplace=True)
tt= list(k7_4['s_cat_id'])
r1=[]
r2=[]
r3=[]
r4=[]
for i in range(len(tt)):
    r1.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_id'])))[0])
    r2.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_value'])))[0])
    r3.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_id'])))[0])
    r4.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_value'])))[0])
k7_4['b_cat_id']=r1
k7_4['b_cat_value']=r2
k7_4['m_cat_id']=r3
k7_4['m_cat_value']=r4

k7_4=k7_4[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

k7_4.to_excel(r'C:\Users\Administrator\Desktop\任务六（七）.xlsx')

## 八、有卖巧克力（平均订单时间<3个月）并且卖口香糖（平均下单时间>3个月）的门店对应前10个POI的巧克力累计销量

1)	求出卖巧克力的门店

In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

2)	求出巧克力平均下单时间小于3个月的门店

In [ ]:
# 把Chocolate的那部分分成下单周期小于3个月和下达周期大于3个月两类
Cstore_id_da3 = [] # 用来储存买Chocolate的那部分下单周期大于3个月的'store_code'
Cstore_id_xiao3 = [] # 用来储存买Chocolate的那部分下单周期小于等于3个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da3.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            Cstore_id_da3.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao3.append(Chocolate_storeid[i]) 

3)	求出（2）中门店中卖口香糖的门店

In [ ]:
# 在Cstore_id_xiao3里面筛选出卖口香糖的门店；
demo1=df1.loc[df1['store_code'].isin(Cstore_id_xiao3)]
demo2=demo1.loc[demo1['ProductCategory']=='Chewing/Bubble Gum']
Chox3_Che=list(set(list(demo2['store_code'])))

4)	求出（3）中口香糖平均下单时间大于3个月的门店；

In [ ]:
# 将Chox3_Che划分为口香糖平均下单时间>3个月
CC_da3=[]
CC_xiao3=[]
for i in range(len(Chox3_Che)):
    
    demo0 = df1.loc[df1['store_code']==Chox3_Che[i]]
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chewing/Bubble Gum']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    print('day',day)
    #计算订单个数
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    print('order_sum',order_sum)
    
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        CC_da3.append(Chox3_Che[i])
    else:
        interval = day/(order_sum-1)
        if interval>90:
            CC_da3.append(Chox3_Che[i])
        else:
            CC_xiao3.append(Chox3_Che[i]) 

5)	求出（4）中门店对应的POI，去重、去充电宝，按照距离有小到大保留前十个POI；

In [ ]:
# CC_da3对应的poi
CC_da3_POI=df_POI[df_POI['storecode'].isin(CC_da3)]
# 去充电宝
CC_da3_POI=CC_da3_POI.drop(CC_da3_POI[CC_da3_POI['s_cat_value']=='充电宝'].index)
# 对每个门店的poi去重（相同POI留下距离近的那个POI）
CC_da3_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k8_1 = CC_da3_POI.groupby(['storecode','s_cat_id']).head(10)

6)	求出（3）中每个门店的巧克力销量（case-ordered）

In [ ]:
# 在Chox3_Che里面筛选出卖巧克力的门店；
demo1=df1.loc[df1['store_code'].isin(Chox3_Che)]
demo2=demo1.loc[demo1['ProductCategory']=='Chocolate']
Chox3_Che_Cho=list(set(list(demo2['store_code'])))
# 门店巧克力销售加总
cho_sum=[] 
for i in range(len(Chox3_Che_Cho)):
    demo3 = df1.loc[df1['store_code']==Chox3_Che_Cho[i]]
    sum_1 = demo3.loc[demo3['ProductCategory']=='Chocolate','case_ordered'].sum()
    cho_sum.append(sum_1)

7)	将（6）中的每个门店销量赋值到（5）中每个门店对应的POI上；

In [ ]:
# k8_1赋上销量信息
# 构建门店编号及其销量信息的表
data3={
    'Chox3_Che_Cho':Chox3_Che_Cho,
    'cho_sum':cho_sum
}
data3=pd.DataFrame(data3)
data3.sort_values(by='Chox3_Che_Cho',inplace=True)

k8_1.sort_values(by='storecode',inplace=True)

sum_2=list(data3['cho_sum'])
s_id =list(data3['Chox3_Che_Cho'])

sum_3 =[]
for i in range(len(s_id)):
    cc=len(k8_1.loc[k8_1['storecode']==s_id[i]])
    for j in range(cc):
        sum_3.append(sum_2[i])

# print(len(sum_3))
# print(len(k3_1))

k8_1['总销量']=sum_3

fff=k8_1[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

8)	求每个POI的销量累计值，按照销量由大到小排序；

In [ ]:
k8_4 = fff.groupby(['s_cat_id','s_cat_value'])['总销量'].sum()
k8_4 = k8_4.reset_index()
k8_4.sort_values(['总销量'],ascending=[0],inplace=True)
tt= list(k8_4['s_cat_id'])
r1=[]
r2=[]
r3=[]
r4=[]
for i in range(len(tt)):
    r1.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_id'])))[0])
    r2.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_value'])))[0])
    r3.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_id'])))[0])
    r4.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_value'])))[0])
k8_4['b_cat_id']=r1
k8_4['b_cat_value']=r2
k8_4['m_cat_id']=r3
k8_4['m_cat_value']=r4

k8_4=k8_4[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

k8_4.to_excel(r'C:\Users\Administrator\Desktop\任务六（八）.xlsx')